The 52-Fragment Golden Gate Assembly using BsaI-HF®v2 is obtained from the New England Biolabs (NEB).
https://international.neb.com/protocols/2018/10/02/golden-gate-assembly-protocol-for-using-neb-golden-gate-assembly-mix-e1601


In [1]:
from logging import Logger
import os
import logging
import sbol3
import labop
import tyto
import uml
import json
import rdflib as rdfl
from typing import Dict, Tuple
from sbol3 import Document

from labop.execution_engine import ExecutionEngine
from labop_convert.markdown.markdown_specialization import MarkdownSpecialization
from labop_convert.opentrons.opentrons_specialization import REVERSE_LABWARE_MAP, OT2Specialization

from IPython.display import Image

# %load_ext autoreload
# %autoreload 2

/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/sbol3/object.py:70: UserWarning: Using a default namespace. To set a namespace use set_namespace()
  warnings.warn('Using a default namespace.'


In [2]:
# set up the document
print('Setting up document')
doc = sbol3.Document()
sbol3.set_namespace('https://bmey3labautomation.com/')

Setting up document


In [3]:
# Import the primitive libraries
print('Importing libraries')
labop.import_library('liquid_handling')
print('... Imported liquid handling')
labop.import_library('plate_handling')
print('... Imported plate handling')
labop.import_library('spectrophotometry')
print('... Imported spectrophotometry')
labop.import_library('sample_arrays')
print('... Imported sample arrays')
labop.import_library('pcr')
print('... Imported pcr')

Importing libraries
... Imported liquid handling
... Imported plate handling
... Imported spectrophotometry
... Imported sample arrays
... Imported pcr


In [4]:
protocol = labop.Protocol('labop_golden_gate_assembly')
protocol.name = "LabOP Golden Gate Assembly Protocol"
doc.add(protocol)

# create the materials to be provisioned
CONT_NS = rdfl.Namespace('https://sift.net/container-ontology/container-ontology#')
OM_NS = rdfl.Namespace('http://www.ontology-of-units-of-measure.org/resource/om-2/')

PREFIX_MAP = json.dumps({"cont": CONT_NS, "om": OM_NS})

In [5]:
# Define reagents
# do we need to define all our reagents? if yes, how do we know which URI for our reagents?
nfh2o = sbol3.Component('nfH2O', 'nucleasefreeh2o')
nfh2o.name = 'Nuclease-free H2O'
doc.add(nfh2o)

t4_dnaligase = sbol3.Component('t4_dnaligase', 't4dnaligasebuffer')
t4_dnaligase.name = 'T4 DNA Ligase Buffer'
doc.add(t4_dnaligase)

dest_plasmid = sbol3.Component('dest_plasmid', 'destinationplasmid')
dest_plasmid.name = 'Destination Plasmid'
doc.add(dest_plasmid)

dna_frag1 = sbol3.Component('dna_frag1', 'dnafragments1')
dna_frag1.name = 'DNA Fragment 1'
doc.add(dna_frag1)

dna_frag2 = sbol3.Component('dna_frag2', 'dnafragments2')
dna_frag2.name = 'DNA Fragment 2'
doc.add(dna_frag2)

gg_enzyme = sbol3.Component('gg_enzyme', 'ggenzymemix')
gg_enzyme.name = 'Golden Gate Enzyme Mix'
doc.add(gg_enzyme)

In [6]:
# Define labware
spec_reagent_plate = labop.ContainerSpec('reagent_plate',
                                    name='Well plate for reagents',
                                    queryString=REVERSE_LABWARE_MAP['nest_96_wellplate_200ul_flat'],
                                    prefixMap=PREFIX_MAP)
spec_destination_plate = labop.ContainerSpec('destination_plate',
                                        name='Destination plate',
                                        queryString=REVERSE_LABWARE_MAP['biorad_96_wellplate_200ul_pcr'],
                                        prefixMap=PREFIX_MAP)
spec_tiprack = labop.ContainerSpec('tiprack',
                                   name='tiprack',
                                   queryString=REVERSE_LABWARE_MAP['opentrons_96_tiprack_10ul'],
                                   prefixMap=PREFIX_MAP)
p20 = sbol3.Agent('p20_single_gen2',
                  name='P20 Single GEN2')
thermocycler = sbol3.Agent('thermocycler_module',
                           name = 'Thermocycler Module')
doc.add(spec_tiprack)
doc.add(spec_reagent_plate)
doc.add(spec_destination_plate)
doc.add(p20)
doc.add(thermocycler)

tiprack = protocol.primitive_step('EmptyContainer',
                                  specification=spec_tiprack)
reagent_plate = protocol.primitive_step('EmptyContainer',
                                        specification=spec_reagent_plate)
destination_plate = protocol.primitive_step('EmptyContainer',
                                            specification=spec_destination_plate)

In [7]:
# Load OT2 robot with labware
left_pipette = protocol.primitive_step('ConfigureRobot', 
                                      #  instrument=OT2Specialization.EQUIPMENT['p20_single_gen2'],
                                      instrument=p20,
                                       mount='left')
thermocycler_module = protocol.primitive_step('ConfigureRobot',
                                              instrument=thermocycler,
                                            #   instrument=OT2Specialization.EQUIPMENT['thermocycler'],
                                              mount='7')
tiprack = protocol.primitive_step('LoadRackOnInstrument',
                                  rack=spec_tiprack,
                                  coordinates='1')
load_reagent_plate = protocol.primitive_step('LoadRackOnInstrument',
                                             rack=spec_reagent_plate,
                                             coordinates='4')
load_destination_plate = protocol.primitive_step('LoadContainerOnInstrument',
                                                 specification=spec_destination_plate,
                                                 instrument=thermocycler,
                                                #  instrument=OT2Specialization.EQUIPMENT['thermocycler'],
                                                 slots='A1:H12') # 96 wells

In [8]:
# Set up target samples
assembly_reaction = protocol.primitive_step('PlateCoordinates',
                                            source=destination_plate.output_pin('samples'),
                                            coordinates='A1:H1')
negative_control = protocol.primitive_step('PlateCoordinates',
                                           source=destination_plate.output_pin('samples'),
                                           coordinates='A2:H2')

In [9]:
# Set up reagents - assuming reagents are loaded onto reagent plate manually
nfh2o_well = protocol.primitive_step('PlateCoordinates',
                                     source=reagent_plate.output_pin('samples'),
                                     coordinates='A1')
t4_dnaligase_well = protocol.primitive_step('PlateCoordinates',
                                            source=reagent_plate.output_pin('samples'),
                                            coordinates='A2')
dest_plasmid_well = protocol.primitive_step('PlateCoordinates',
                                            source=reagent_plate.output_pin('samples'),
                                            coordinates='A3')
dna_frag1_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A4')
dna_frag2_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A5')
gg_enzyme_well = protocol.primitive_step('PlateCoordinates',
                                         source=reagent_plate.output_pin('samples'),
                                         coordinates='A6')

In [10]:
# protocol
transfer = protocol.primitive_step('Transfer',
                                   source=nfh2o_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(20,tyto.OM.microliter))
transfer.name = '1. Transfer 28 ul of nuclease-free H2O (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=nfh2o_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(8,tyto.OM.microliter))
transfer.name = '1. Transfer 28 ul of nuclease-free H2O (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=nfh2o_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   amount=sbol3.Measure(20,tyto.OM.microliter))
transfer.name = '2. Transfer 30 ul of nuclease-free H2O (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=nfh2o_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   amount=sbol3.Measure(10,tyto.OM.microliter))
transfer.name = '2. Transfer 30 ul of nuclease-free H2O (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=t4_dnaligase_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(4,tyto.OM.microliter))
transfer.name = '3. Transfer 4 ul of T4 DNA ligase buffer (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=t4_dnaligase_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   amount=sbol3.Measure(4,tyto.OM.microliter))
transfer.name = '4. Transfer 4 ul of T4 DNA ligase buffer (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=dest_plasmid_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '5. Transfer 2 ul of destination plasmid (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=dest_plasmid_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '6. Transfer 2 ul of destination plasmid (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag1_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '7. Transfer 2 ul of DNA fragment 1 (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag1_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '8. Transfer 2 ul of DNA fragment 1 (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag2_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '9. Transfer 2 ul of DNA fragment 2 (assembly reaction).'

transfer = protocol.primitive_step('Transfer',
                                   source=dna_frag2_well.output_pin('samples'),
                                   destination=negative_control.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '10. Transfer 2 ul of DNA fragment 2 (negative control).'

transfer = protocol.primitive_step('Transfer',
                                   source=gg_enzyme_well.output_pin('samples'),
                                   destination=assembly_reaction.output_pin('samples'),
                                   amount=sbol3.Measure(2,tyto.OM.microliter))
transfer.name = '11. Transfer 2 ul of Golden Gate enzyme mix (assembly reaction).'


# mix

# incubate

In [11]:
filename = 'ot2_gg_labop'
# filename = os.path.join(out_dir, "ot2_gibson_assembly_labop") # out_dir currently points to test
agent = sbol3.Agent('ot2_machine', name='OT2 machine')
ee = ExecutionEngine(specializations=[OT2Specialization(filename)])
parameter_values = []
execution = ee.execute(protocol, agent, id='test_execution')

ERROR:/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/labop/execution_engine_utils.py:Could Not Process https://bmey3labautomation.com/test_execution/CallBehaviorExecution17: 'pipette'
ERROR:/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/labop/execution_engine_utils.py:Could Not Process https://bmey3labautomation.com/test_execution/CallBehaviorExecution18: 'pipette'
ERROR:/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/labop/execution_engine_utils.py:Could Not Process https://bmey3labautomation.com/test_execution/CallBehaviorExecution19: 'pipette'
ERROR:/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/labop/execution_engine_utils.py:Could Not Process https://bmey3labautomation.com/test_execution/CallBehaviorExecution20: 'pipette'
ERROR:/Users/jeytse/BMEY3_LabAutomationGroupProject/labOP_env/lib/python3.10/site-packages/labop/execution_engin

Successful execution. Script dumped to ot2_gg_labop.
